In [1]:
%load_ext dotenv
%dotenv

In [2]:
import os
from pydantic import BaseModel, Field
from openai import OpenAI
from typing import Optional, List
import json
from utils import neo4j_driver

client = OpenAI()

/Users/tomazbratanic/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [3]:
class Location(BaseModel):
    """
    Represents a physical location including address, city, state, and country.
    """

    address: Optional[str] = Field(
        ..., description="The street address of the location."
    )
    city: Optional[str] = Field(..., description="The city of the location.")
    state: Optional[str] = Field(
        ..., description="The state or region of the location."
    )
    country: str = Field(
        ...,
        description="The country of the location. Use the two-letter ISO standard.",
    )


class Organization(BaseModel):
    """
    Represents an organization, including its name and location.
    """

    name: str = Field(..., description="The name of the organization.")
    location: Location = Field(
        ..., description="The primary location of the organization."
    )
    role: str = Field(
        ...,
        description="The role of the organization in the contract, such as 'provider', 'client', 'supplier', etc.",
    )

contract_types = [
    "Service Agreement",
    "Licensing Agreement",
    "Non-Disclosure Agreement (NDA)",
    "Partnership Agreement",
    "Lease Agreement"
]

class Contract(BaseModel):
    """
    Represents the key details of the contract.
    """

    contract_type: str = Field(
        ...,
        description="The type of contract being entered into.",
        enum=contract_types,
    )
    parties: List[Organization] = Field(
        ...,
        description="List of parties involved in the contract, with details of each party's role.",
    )
    effective_date: str = Field(
        ...,
        description="The date when the contract becomes effective. Use yyyy-MM-dd format.",
    )
    term: str = Field(
        ...,
        description="The duration of the agreement, including provisions for renewal or termination.",
    )
    contract_scope: str = Field(
        ...,
        description="Description of the scope of the contract, including rights, duties, and any limitations.",
    )
    end_date: Optional[str] = Field(
        ...,
        description="The date when the contract becomes expires. Use yyyy-MM-dd format.",
    )
    total_amount: Optional[float] = Field(
        ..., description="Total value of the contract."
    )
    governing_law: Optional[Location] = Field(
        ..., description="The jurisdiction's laws governing the contract."
    )

In [4]:
system_message = """
You are an expert in extracting structured information from legal documents and contracts.
Identify key details such as parties involved, dates, terms, obligations, and legal definitions.
Present the extracted information in a clear, structured format. Be concise, focusing on essential
legal content and ignoring unnecessary boilerplate language."""

In [5]:
def extract(document, model="gpt-4o-2024-08-06", temperature=0):
    response = client.beta.chat.completions.parse(
        model=model,
        temperature=temperature,
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content": document},
        ],
        response_format=Contract,
    )
    return json.loads(response.choices[0].message.content)

In [6]:
# Read the file
with open('../data/license_agreement.txt', 'r') as file:
    contents = file.read()

In [7]:
#print(contents)

In [8]:
data = extract(contents)
data

{'contract_type': 'Licensing Agreement',
 'parties': [{'name': 'Mortgage Logic.com, Inc.',
   'location': {'address': 'Two Venture Plaza, 2 Venture',
    'city': 'Irvine',
    'state': 'California',
    'country': 'US'},
   'role': 'Client'},
  {'name': 'TrueLink, Inc.',
   'location': {'address': '3026 South Higuera',
    'city': 'San Luis Obispo',
    'state': 'California',
    'country': 'US'},
   'role': 'Provider'}],
 'effective_date': '1999-02-26',
 'term': '1 year, with automatic renewal for successive one-year periods unless terminated',
 'contract_scope': 'TrueLink grants Mortgage Logic.com a non-exclusive license to use the Interface for origination, underwriting, processing, and funding of consumer finance receivables. TrueLink will provide hosting services, including storage, response time management, bandwidth, availability, access, backups, internet connection, domain name assistance, and transmission of credit data. TrueLink will also provide support services with at lea

In [9]:
neo4j_driver.execute_query(
    "CREATE CONSTRAINT IF NOT EXISTS FOR (c:Contract) REQUIRE c.id IS UNIQUE;"
)
neo4j_driver.execute_query(
    "CREATE CONSTRAINT IF NOT EXISTS FOR (o:Organization) REQUIRE o.name IS UNIQUE;"
)
neo4j_driver.execute_query(
    "CREATE CONSTRAINT IF NOT EXISTS FOR (l:Location) REQUIRE l.fullAddress IS UNIQUE;"
)

EagerResult(records=[], summary=<neo4j._work.summary.ResultSummary object at 0x137fa7950>, keys=[])

In [10]:
import_query = """WITH $data AS contract_data
// Create Contract node
MERGE (contract:Contract {id: randomUUID()})
SET contract += {
  contract_type: contract_data.contract_type,
  effective_date: contract_data.effective_date,
  term: contract_data.term,
  contract_scope: contract_data.contract_scope,
  end_date: contract_data.end_date,
  total_amount: contract_data.total_amount,
  governing_law: contract_data.governing_law.state + ' ' +
                 contract_data.governing_law.country
}
WITH contract, contract_data
// Create Party nodes and their locations
UNWIND contract_data.parties AS party
MERGE (p:Organization {name: party.name})
MERGE (loc:Location {
  fullAddress: party.location.address + ' ' +
                party.location.city + ' ' +
                party.location.state + ' ' +
                party.location.country})
SET loc += {
  address: party.location.address,
  city: party.location.city,
  state: party.location.state,
  country: party.location.country
}
// Link party to their location
MERGE (p)-[:LOCATED_AT]->(loc)
// Link parties to the contract
MERGE (p)-[r:HAS_PARTY]->(contract)
SET r.role = party.role
"""
neo4j_driver.execute_query(import_query, data=data)

EagerResult(records=[], summary=<neo4j._work.summary.ResultSummary object at 0x137fdd210>, keys=[])